In [1]:
# Imports
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# URL de base
url = "https://www.af3v.org/resultats-des-recherches/?stv%5B1%5D=on&sact%5B9%5D=on&sact%5B6%5D=on&ssort=dista"
# Tri : voie verte lisse, vélo de route et roller

In [3]:
# Récupération des noms des fiches VV et les liens sur les différentes pages de résultats 

data = {}
index = 0

for i in range(0,321,10):
    link = f"https://www.af3v.org/resultats-des-recherches/?stv%5B1%5D=on&sact%5B9%5D=on&sact%5B6%5D=on&ssort=dista&searchoffs={i}&blockOrLine=0"

    html = requests.get(link)

    # récupérer le nom, les villes de début et fin et le lien de la fiche
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        cards = [div for div in soup.find_all("div", class_="openSansReg temCom198")] # Pour chaque élément div trouvé dans soup.find_all(...), il est gardé dans une liste.
        cards_town = [div for div in soup.find_all("div", class_="openSansReg s12w400 coc7c7c7")]
    

        for card, cardt in zip(cards, cards_town):      # si le nombre de résultats change surtout pour dernière page)
            var_1 = card.find('a')
            if var_1:
                name = var_1.get_text(strip=True)
                link_end = var_1['href']
                link_full = "https://www.af3v.org" + link_end
                trajet = cardt.get_text(strip=True)
                words = re.search(r'De (.+?) à (.+)', trajet)
                

                if words:
                    start = words.group(1)
                    end = words.group(2)

                    data[index] = { 'Nom': name,
                                    'Lien': link_full,
                                    'Début': start,
                                    'Fin': end}
                    index += 1
        
               

                    

In [ ]:
data

In [4]:
 # récupérer les autres infos caractéristiques en rentrant dans la fiche
for i in data:
    link_fiche = data[i]['Lien']
    html_fiche = html = requests.get(link_fiche)

    if html_fiche.status_code == 200:
        soup_fiche = BeautifulSoup(html_fiche.text, 'html.parser')
        caracteristics_dist = [div for div in soup_fiche.find_all("div", class_="co767676 openSansMed s14w400")]
        if len(caracteristics_dist) >= 4:
            distance = caracteristics_dist[0].get_text(strip=True)
            type_voie = caracteristics_dist[1].get_text(strip=True)
            nature = caracteristics_dist[2].get_text(strip=True)
            revetement = caracteristics_dist[3].get_text(strip=True)
            data[i].update({ 'Distance': distance,
                        'Type voie': type_voie,
                        'Nature voie': nature,
                        'Revêtement': revetement})
                            


In [ ]:
data

In [5]:
for i in data:
    link_fiche = data[i]['Lien']
    html_fiche = html = requests.get(link_fiche)

    if html_fiche.status_code == 200:
        soup_fiche = BeautifulSoup(html_fiche.text, 'html.parser')
        car_map = [div for div in soup_fiche.find_all("iframe", id="carte_voies")]
        for map in car_map:
            carte = map["data-url"]
            coords_all = re.search(r'(?<=bbox=)[^&]*', carte) # cherche tout ce qui est après = et avant &
            if coords_all:
                coords_str = coords_all.group() # récupérer la str extraite
                coords = coords_str.split(',') # séparer par ,
                coord_start = (coords[0], coords[1])
                coord_end = (coords[2], coords[3])
                data[i].update({ 'Carte': carte,
                                'Coordonnées de début': coord_start,
                                'Coordonnées de fin': coord_end})
                            


In [8]:
data

{0: {'Nom': 'Voie Verte du Chemin d’Encochebérot',
  'Lien': 'https://www.af3v.org/les-voies-vertes/voies/1276-voie-verte-du-chemin-d-encocheberot',
  'Début': 'L ISLE JOURDAIN',
  'Fin': 'L ISLE JOURDAIN',
  'Distance': '1,6km',
  'Type voie': 'Voie verte lisse',
  'Nature voie': 'voie verte',
  'Revêtement': 'Enrobé',
  'Carte': 'https://sig.af3v.org/index.php/view/embed/?repository=rep1&project=veloroutes&bbox=1.09673702748082,43.62109781579031,1.1131950619939002,43.6267673096849&crs=EPSG:4326',
  'Coordonnées de début': ('1.09673702748082', '43.62109781579031'),
  'Coordonnées de fin': ('1.1131950619939002', '43.6267673096849')},
 1: {'Nom': 'Voie Verte du canal de Saint-Martory',
  'Lien': 'https://www.af3v.org/les-voies-vertes/voies/1246-voie-verte-du-canal-de-saint-martory',
  'Début': 'CUGNAUX',
  'Fin': 'TOURNEFEUILLE',
  'Distance': '2,1km',
  'Type voie': 'Voie verte lisse',
  'Nature voie': 'voie verte',
  'Revêtement': 'Enrobé',
  'Carte': 'https://sig.af3v.org/index.php/v

In [6]:
# mettre les infos en df
df = pd.DataFrame.from_dict(data, orient='index')

In [7]:
import ast

def convert_en_df(data):
    df = pd.DataFrame.from_dict(data, orient='index')

    def midpoint(lat1, lon1, lat2, lon2):
        mid_lat = (lat1 + lat2) / 2
        mid_lon = (lon1 + lon2) / 2
        return mid_lat, mid_lon

    def calculate_midpoint_voie_verte(row):
        try:
            coord_debut = row['Coordonnées de début']
            coord_fin = row['Coordonnées de fin']
            
            # Convertir selon le type
            if isinstance(coord_debut, str):
                coord_debut = coord_debut.strip('()')
                lat_debut, lon_debut = map(float, coord_debut.split(','))
            else:
                lat_debut, lon_debut = float(coord_debut[0]), float(coord_debut[1])
            
            if isinstance(coord_fin, str):
                coord_fin = coord_fin.strip('()')
                lat_fin, lon_fin = map(float, coord_fin.split(','))
            else:
                lat_fin, lon_fin = float(coord_fin[0]), float(coord_fin[1])
            
            return midpoint(lat_debut, lon_debut, lat_fin, lon_fin)
        except Exception as e:
            print(f"Erreur: {e}")
            return None
    
    df['Coordonnées milieu'] = df.apply(calculate_midpoint_voie_verte, axis=1)

    def safe_literal_eval(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"Erreur pour la valeur {val}: {e}")
                return None
        else:
            return val  # déjà un tuple ou autre type valide
    
    df["Coordonnées milieu"] = df["Coordonnées milieu"].apply(safe_literal_eval)
    df["Coordonnées de début"] = df["Coordonnées de début"].apply(safe_literal_eval)
    df["Coordonnées de fin"] = df["Coordonnées de fin"].apply(safe_literal_eval)
    df['Distance'] = df['Distance'].str.replace('km', '', regex=False).str.replace(',', '.').astype(float)
    df['Début'] = df['Début'].str.capitalize()
    df['Fin'] = df['Fin'].str.capitalize()
    df['% Couverture correspondant (200m)'] = round((200 / (df['Distance'] * 1000) * 100),2)

    return df

df = convert_en_df(data)

In [8]:
def categorize_nature(voie) -> str :
    voie = str(voie).lower()
    if 'rivière' in voie or 'canal' in voie or 'chemin de halage' in voie:
        return 'Bord de rivière ou canal'
    elif 'sentier' in voie or 'chemin' in voie:
        return 'Bord de sentier'
    elif 'route' in voie:
        return 'Bord de route'
    elif 'piste' in voie:
        return 'Piste cyclable'
    elif 'chemin de halage' in voie:
        return 'Chemin de halage'
    elif 'ancienne' in voie:
        return 'Ancienne ligne ferrovière'
    elif 'verte' in voie:
        return 'Voie verte'
    elif 'ferrée'in voie:
        return 'Bord de voie ferrée'
    else:
        return 'Voie verte'
    
df['Groupes nature voie'] = df['Nature voie'].apply(categorize_nature)

In [9]:
df.to_csv("df_voies_vertes.csv")

In [25]:
# Imports pour viz cartographie

import folium
from typing import Optional
from folium.plugins import MarkerCluster

In [11]:
# Définition de la fonction pour trouver les  coordonnées

url_map = 'http://api-adresse.data.gouv.fr/search/'

def get_lat_lon(address : str):
    params = {
        'q': address,

        'limit': 1
    }
    response = requests.get(url_map, params=params)
    j = response.json()
    if len(j.get('features')) > 0:
        first_result = j.get('features')[0]
        lon, lat = first_result.get('geometry').get('coordinates')
        return(lat, lon)
    else:
        return('No result')

In [27]:
#Récupérer les coordonnées de France à l'aide de la fonction get_lat_lon
france_coord = get_lat_lon("France")

In [28]:
#Centrer la carte folium sur ces coordonnées
my_map = folium.Map(location=france_coord, zoom_start=6)

In [29]:
#Fonction pour placer les marqueurs sur la carte
def add_markers(row, my_map=my_map):
    location = row["Nom"]
    lon, lat = row["Coordonnées milieu"]
    folium.Marker(location=(lat, lon), popup=location, icon=folium.Icon(icon="leaf", color="green")).add_to(my_map)
    return f"Marker added for {location}"

#Ajouter les marqueurs sur la carte
df.apply(add_markers, axis=1)

#Afficher la carte folium
my_map